# Experimenting With tkinter

It's possible to create fast, lightweight canvases using tkinter. By construction, canvases open in a separate window. In this folder we experiment with embedding this canvas in a cell, the way that iPyCanvas does.

tkinter canvases support styling options for drawing such as line color , marker width, and canvas background color. I suggest locking these options so the user writes with a relatively narrow (width<5) black marker on a white canvas to best suit the OCR.

Other code snippets explored here are related to extracting an image from tkinter canvas.

Embedding tkinter is an ongoing process. tkinter operates in an independent window that serves as a canvas where drawing takes place. We must take this operation and insert it ina cell. ipycanvas works in this manner, but there is no interoperability between the tkinter class created to run the canvas and ipycanvas' method to populate a canvas inside a cell. See **Embedding in a cell** below to find the latest state of my thoughts on what could happen to make this work. 

In [2]:
from tkinter import Tk, Canvas, ttk, Button
from tkinter import constants as con

import PIL
from PIL import ImageGrab, ImageTk, ImageDraw

import random
import string

## A Simple Canvas

The code below outputs a canvas in a separate window.

In [2]:
def savePosn(event):
    global lastx, lasty
    lastx, lasty = event.x, event.y

def addLine(event):
    canvas.create_line((lastx, lasty, event.x, event.y))
    savePosn(event)

root = Tk()
root.columnconfigure(0, weight=1)
root.rowconfigure(0, weight=1)

canvas = Canvas(root)
canvas.grid(column=0, row=0, sticky=(con.N, con.W, con.E ,con.S))
canvas.bind("<Button-1>", savePosn)
canvas.bind("<B1-Motion>", addLine)

root.mainloop(0)

### Output 1:

The root object with the mainloop() method will open a window where drawing takes place.

<img src="./figures/tkinter_canvas_1.png" alt="Kitten" title="A cute kitten" width="350" height="100" /> 


## Another Canvas

This canvas encapsulates the code above.

In [7]:
class Sketchpad(Canvas):
    def __init__(self, parent, **kwargs):
        super().__init__(parent, **kwargs)
        self.bind("<Button-1>", self.save_posn)
        self.bind("<B1-Motion>", self.add_line)
        
    def save_posn(self, event):
        self.lastx, self.lasty = event.x, event.y

    def add_line(self, event):        
        self.create_line((self.lastx, self.lasty, event.x, event.y),
                         fill="black", width=1)
        self.save_posn(event)

        
root = Tk()
root.columnconfigure(0, weight=1)
root.rowconfigure(0, weight=1)

sketch = Sketchpad(root,background="white")
sketch.grid(column=0, row=0, sticky=(con.N, con.W, con.E, con.S))

root.mainloop()

### Output 2:

With the default options.

<img src="./figures/tkinter_canvas_2.png" alt="" title="" width="350" height="100" /> 

### Output 3:

Playing with multiple options can be done like this:  

`self.create_line((self.lastx, self.lasty, event.x, event.y), fill="red", width=5)`

There are issues with wider paint brushes in that they tend to segment.  I think the color black with a width <5 is best so the OCR receives quality data.  Background color can alos play a role. Best set this to white so there is a higher contrast between writing (black) and canvas (white).

<img src="./figures/tkinter_canvas_3.png" alt="" title="" width="350" height="100" /> 

<img src="./figures/tkinter_canvas_4.png" alt="" title="" width="350" height="100" /> 

<img src="./figures/tkinter_canvas_5.png" alt="" title="" width="350" height="100" /> 

### Output 4:

Tkinter canvas with a white background.

`sketch = Sketchpad(root,background="white")`

<img src="./figures/tkinter_canvas_6.png" alt="" title="" width="350" height="100" /> 

## Extracting the tkinter Canvas Image

Working with tkinter, we employ code to capture the writing on the canvas widget.

### Working Basic Hello World Application tkinter

This application showcases a button that can be used to close a tkinter widget.

In [7]:
import tkinter as tk

class Application(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        self.pack()
        self.create_widgets()

    def create_widgets(self):
        self.hi_there = tk.Button(self)
        self.hi_there["text"] = "Hello World\n(click me)"
        self.hi_there["command"] = self.say_hi
        self.hi_there.pack(side="top")

        self.quit = tk.Button(self, text="QUIT", fg="red",
                              command=self.master.destroy)
        self.quit.pack(side="bottom")

    def say_hi(self):
        print("hi there, everyone!")

In [6]:
root = tk.Tk()
app = Application(master=root)
app.mainloop()

### Output 5:

<img src = "figures/tkinter_canvas_7.png" />

### Saving tkinter Canvas To A File: Part 1

This code shows how to use PIL's ImageGrab to save the tkinter canvas. It doesn't yet work for us because it requires the drawing process to be finished. 

In [8]:
def getter(widget):
    x=root.winfo_rootx()+widget.winfo_x()
    y=root.winfo_rooty()+widget.winfo_y()
    x1=x+widget.winfo_width()
    y1=y+widget.winfo_height()
    ImageGrab.grab().crop((x,y,x1,y1)).save("ex.png")

root = tk.Tk()
cv = Canvas(root)
cv.create_rectangle(10,10,50,50)
cv.pack()

getter(cv)

In [9]:
root.mainloop()

<img src="figures/tkinter_canvas_8.png" width = 300 />

### Saving tkinter Canvas To File: Part 2

This code is from: https://www.youtube.com/watch?v=OdDCsxfI8S0

In [23]:
def rnd_image_filename(N=10):
    filetag="".join(random.choices(string.ascii_uppercase + string.digits, k=N))
    filename="canvas_img_"+filetag+".png"
    return filename

def save():
    filename = rnd_image_filename(N=5)
    canvas_image.save(filename)

def paint(event):
    # python_green = "#476042"
    x1, y1 = (event.x - 1), (event.y - 1)
    x2, y2 = (event.x + 1), (event.y + 1)
    cv.create_oval(x1, y1, x2, y2,fill="black",width=2)
    draw.line([x1, y1, x2, y2],fill="black",width=2)

width = 600
height = 400
center = height//2
white = (255, 255, 255)
background = "white"
green = (0,128,0)

root = Tk()

# Instantiate tkinter create a canvas to draw on.
cv = Canvas(root, width=width, height=height, bg=background)
cv.pack()

# Use PIL to create an empty image and draw object to draw on
# memory only, not visible to the user.
canvas_image = PIL.Image.new("RGB", (width, height), white)
draw = ImageDraw.Draw(canvas_image)

cv.pack(expand=True, fill="both")
cv.bind("<B1-Motion>", paint)

'4595397248paint'

In [24]:
# Create a button to save the image.  Binds to the save function.
button = Button(text="save",command=save)
button.pack()
root.mainloop()

<img src=figures/tkinter_canvas_9.png width=500 />

### Embedding tkingter In A Cell

From the documentation, it looks like only an ipycanvas widget will work well here. 
It remains to be seen if tkinter can be instantiated inside a cell. 

We can explore the ipycanvas code for inspiration. The ipycanvas code, replicated here below.

tkinter embeds a canvas in a cell by using its ipywidget `HBox`.
Below are the imports for ipycanvas.

In [1]:
from ipywidgets import Image, ColorPicker, IntSlider, link, AppLayout, HBox
from ipycanvas import RoughCanvas, hold_canvas

ipycanvas uses an object called RoughCanvas (type = <class 'ipycanvas.canvas.RoughCanvas'>) as the actual canvas widget the user draws on. 

Further down **HBox** (type = <class 'ipywidgets.widgets.widget_box.HBox'>) is applied to a tuple of widgets, which the user can interact with to style the drawing of the canvas. For now, the option below only admits the canvas and no other widgets. 

**HBox** is a cell container for the RoughCanvas. This what the user sees immediately below a cell to draw in. I'm thinking this can be modified to use tkinter's canvas instead, in lieu of finding a solution that exists for tkinter exclusively, which I don't exactly see.

In [8]:
def on_mouse_down(x, y):
    global drawing
    global position
    global shape

    drawing = True
    position = (x, y)
    shape = [position]

def on_mouse_move(x, y):
    global drawing
    global position
    global shape

    if not drawing:
        return

    with hold_canvas(canvas):
        canvas.stroke_line(position[0], position[1], x, y)
        position = (x, y)

    shape.append(position)

def on_mouse_up(x, y):
    global drawing
    global position
    global shape

    drawing = False
    
    with hold_canvas(canvas):
        canvas.stroke_line(position[0], position[1], x, y)

# Use these settings to modify the canvase size
width = 800
height = 200

canvas = RoughCanvas(width=width, height=height)

drawing = False
position = None
shape = []

canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)
canvas.stroke_style = "#749cb8"
canvas.line_width = 5.0

picker = ColorPicker(description="Color:", value="black")
link((picker, "value"), (canvas, "stroke_style"))
link((picker, "value"), (canvas, "fill_style"))

In [ ]:
# HBox that actually draws the widget being used.
HBox((canvas,))

<img src="figures/tkinter_canvas_10.png" width=500 />